In [1]:
import os

In [2]:
os.chdir("..")

In [3]:
%pwd

'/home/lucky/Documents/projects/MyProject/smoke_detection'

In [4]:
from google.cloud import storage
from tqdm import tqdm

In [5]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")



# Set your Google Cloud Storage bucket name
bucket_name = "smoke_detection"
# Set the source blob name (file to download from the bucket)
source_blob_name = "smoke_data.zip"
# Set the destination file name (local file to save the downloaded data)
destination_file_name = "smoke_data.zip"

# Call the function to download the file with tqdm progress bar
download_blob(bucket_name, source_blob_name, destination_file_name)

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
key_file_path = "/home/lucky/Documents/key_service_account.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_file_path

In [9]:
from dataclasses import dataclass
from pathlib import Path

In [10]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    bucket_name: str
    source_blob_name: str
    local_data_file: Path
    unzip_dir: Path

### Data Ingestion Configuration Manager

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
CONFIG_FILE_PATH, PARAMS_FILE_PATH

(PosixPath('config/config.yaml'), PosixPath('params.yaml'))

In [13]:
class ConfigurationManager:
    """
        responsible for creating the Directories needed for data
        and return entity -> DataIngestionConfig
    """
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            bucket_name=config.bucket_name,
            source_blob_name=config.source_blob_name,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

### Data Ingestion Class

In [14]:
import os
import urllib.request as request
import ssl
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        storage_client = storage.Client()

        bucket = storage_client.bucket(self.config.bucket_name)
        blob = bucket.blob(self.config.source_blob_name)

        
        if not os.path.exists(self.config.local_data_file):
            blob.download_to_filename(self.config.local_data_file)
            logger.info(f"Blob {source_blob_name} downloaded to {self.config.local_data_file}.")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-29 23:15:47,471: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-29 23:15:47,473: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-29 23:15:47,474: INFO: common: created directory at: artifacts]
[2023-12-29 23:15:47,475: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-29 23:15:47,561: INFO: 3595080592: File already exists of size: ~ 65273 KB]
